In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from graphviz import Digraph
import random

In [2]:
class Error(Exception):
    def __init__(self, expr = None, msg = None):
        self.expr = expr
        self.msg = msg
class inputSmallerThanKernel(Error):
    def __init__(self):
        super(inputSmallerThanKernel, self).__init__()
class nodeDoesNotExist(Error):
    def __init__(self):
        super(nodeDoesNotExist, self).__init__()

In [3]:
class node(object):
    nodes = []
    def __init__(self, input_shape = (0, 0, 0), output_shape = (0, 0, 0)): # c, i1, i2
        node.nodes.append(self)
        self.no  = len(node.nodes)
        self.in_adj = []
        self.out_adj = []
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.compatible = True
            
    def node_alright(self, curr_node):
        try:
            assert(issubclass(type(curr_node), node))
        except:
            raise Error('Not a node')
# Put this section in graph class
#         try:
#             assert(curr_node in graph_nodes)
#         except:
#             raise nodeDoesNotExist
    
    def determine_compatibility(self):
        for curr_node in self.in_adj:
            curr = (curr_node.output_shape == self.input_shape)
            self.compatible  = self.compatible and curr
            
        for curr_node in self.out_adj:
            curr = (curr_node.input_shape == self.output_shape)
            self.compatible = self.compatible and curr

    def describe_adj_list(self, in_adj, out_adj):
        assert isinstance(in_adj, list), 'in_adj must be a list'
        assert isinstance(in_adj, list), 'out_adj must be a list'
        for curr_node in in_adj + out_adj:
            try:
                self.node_alright(curr_node)
            except Exception as e:
                raise e
        self.in_adj = in_adj
        self.out_adj = out_adj

    def out_shape(self):
        pass
    
    def remove(self):
        ## needed in graph class
        pass
    
    
    def __str__(self):
        return str(type(self)) + " " + str(self.no)

    def __repr__(self):
        return self.__str__()

In [4]:
class convolution_block(nn.Module, node):
    all_convs = []
    
    def __init__(self, in_h, in_w, in_channels, out_channels, kernel_size, padding = 0, stride = 1):
        try:
            assert(min(in_h, in_w) +2*padding >= kernel_size)
        except:
            raise inputSmallerThanKernel
        super(convolution_block, self).__init__()
        node.__init__(self, (in_channels, in_h, in_w))
        convolution_block.all_convs.append(self)
        self.in_channels  = in_channels
        self.out_channels = out_channels 
        self.kernel_size = kernel_size
        self.stride = stride 
        self.padding = padding
        self.output_shape = self.out_shape()
        
        # NN Layers
        self.conv_layer = nn.Conv2d(self.in_channels, self.out_channels, self.kernel_size, self.stride, self.padding)
        self.batch_norm = nn.BatchNorm2d(self.out_channels)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.conv_layer(x)
        x = self.batch_norm(x)
        x = self.relu(x)
        return x

    def out_shape(self):
        c, h, w = self.input_shape
        C = self.out_channels
        H = (h + 2*self.padding - self.kernel_size)/self.stride + 1
        W = (w + 2*self.padding - self.kernel_size)/self.stride + 1
        return (C, H, W)
    
#     def determine_compatibility(self):
#         super(convolution_block, self).determine_compatibility()
#         self.compatible  = self.compatible and (len(self.in_adj) == 1)

    def describe_adj_list(self, in_adj, out_adj):
        super(convolution_block, self).describe_adj_list(in_adj, out_adj)
        try:
            assert(len(in_adj) == 1)
        except:
            print(in_adj)
            raise Error('A convolution block can have only one in-edge')
    
    def remove(self): 
        ### Remove from all_convs list
        pass

In [5]:
class max_pool_node(nn.Module, node):
    all_max_pools = []
    
    def __init__(self, in_h, in_w, in_channels, kernel_size, padding = 0, stride = 1):
        try:
            assert(min(in_h, in_w) +2*padding > kernel_size)
        except:
            raise inputSmallerThanKernel
        super(max_pool_node, self).__init__()    
        node.__init__(self, (in_channels, in_h, in_w))
        max_pool_node.all_max_pools.append(self)
        self.kernel_size = kernel_size
        self.stride = stride 
        self.padding = padding
        self.output_shape = self.out_shape()
        
        ## NN Layer
        self.max_pool_layer = nn.MaxPool2d(self.kernel_size, self.stride, self.padding)
        
    def forward(self, x):
        return self.max_pool_layer(x)
    
    def out_shape(self):
        c, h, w = self.input_shape
        C = c
        H = (h + 2*self.padding - self.kernel_size)/self.stride + 1
        W = (w + 2*self.padding - self.kernel_size)/self.stride + 1
        return (C, H, W)
    
#     def determine_compatibility(self):
#         super(max_pool_node, self).determine_compatibility()
#         self.compatible  = self.compatible and (len(self.in_adj) == 1)

    def describe_adj_list(self, in_adj, out_adj):
        super(max_pool_node, self).describe_adj_list(in_adj, out_adj)
        try:
            assert(len(in_adj) == 1)
        except:
            raise Error('A max-pool block can have only one in-edge')

    def remove(self):
        pass

In [6]:
### Is is it required to be to a derived class of nn.Module ?
class merge_node(node):
    all_merge_nodes = []
    
    def __init__(self, parents, child):
#         super(merge_node, self).__init__()
        node.__init__(self)
        try:
            self.describe_adj_list([parents[0], parents[1]], [child])
        except Exception as e:
            raise e
        merge_node.all_merge_nodes.append(self)
        
    def describe_adj_list(self, in_adj, out_adj):
        super(merge_node, self).describe_adj_list(in_adj, out_adj)
        try:
            assert(len(in_adj) == 2)
        except:
            raise Error('Parents must be exactly two')
        
class add_node(nn.Module, merge_node):
    all_add_nodes = []
    
    def __init__(self, parents, child):
        super(add_node, self).__init__()
        merge_node.__init__(self, parents, child)
        add_node.all_add_nodes.append(self)
        self.input_shape = self.in_adj[0].output_shape
        self.output_shape = self.out_shape()
    
    ### Does it allow to input paramteters ?
    def forward(self, x, y):
        return x+y ### Check if their data strcutre type supports this addition
        
    def out_shape(self):
        return self.input_shape
    
class concat_node(merge_node):
    # Define Later
    pass

class convex_merge_node(merge_node):
    # Define Later
    pass

In [45]:
class Network(object):
    def __init__(self):
        self.adj_mat = {}
        self.adj_list = {}
        self.nodes = set()
        self.int_to_node = {}
        self.node_to_int = {}
        self.conv_blocks = []
        self.max_pool_blocks = [] # Change naming conv maybe ?
        self.topsort = []
        self.max_no = 0
        
    def __init__(self, adj_list, int_to_node):
        assert isinstance(int_to_node, dict), 'int_to_node must be a dictionary'
        for _, cnode in int_to_node.items():
            assert isinstance(cnode, node), 'mapping in int_to_node should be to a node'
        
        assert isinstance(adj_list, dict), 'adj_list should be a dictionary'
        assert(len(int_to_node) == len(adj_list))
        for cnode, li in adj_list.items():
            assert cnode in int_to_node, 'mismatch between int_to_node and adj_list'
            try:
                assert(isinstance(li, list))
                assert(len(li) == 2)
                assert(isinstance(li[0], list) and isinstance(li[1], list))
            except:
                raise Error('Each mapping in adj_list should be to a two-dim list')
            for child_node in li[0]:
                assert child_node in int_to_node, 'mismatch between int_to_node and adj_list'
            for child_node in li[1]:
                assert child_node in int_to_node, 'mismatch between int_to_node and adj_list'

        self.adj_list = adj_list
        self.adj_mat = self.get_adj_mat(self.adj_list)
        self.nodes = set(int_to_node.keys())
        self.int_to_node = int_to_node
        self.node_to_int = self.get_node_to_int(self.int_to_node)
        self.max_no = max(self.int_to_node)
        self.conv_blocks, self.max_pool_blocks = self.get_conv_and_max_pool_blocks()
        self.topsort = self.topsorting()
        
#     def __init__(self, random_init):
#         if random_init:
#             # Do random network construction
#             pass
#         else:
#             self.__init__()
    
    def get_node_to_int(self, int_to_node):
        node_to_int = {}
        for no, cnode in int_to_node.items():
            node_to_int[cnode] = no
        return node_to_int

        
    def get_adj_mat(self, adj_list):
        adj_mat = {}
        nodes = adj_list.keys()
        for x in nodes:
            adj_mat[x] = {}
            for y in nodes:
                adj_mat[x][y] = 0
        for cnode, li in adj_list.items():
            for par in li[0]:
                adj_mat[par][cnode] = 1
            for child in li[1]:
                adj_mat[cnode][child] = 1
        return adj_mat
    
    def get_conv_and_max_pool_blocks(self):
        conv_blocks = []
        max_pool_blocks = []
        for x in self.nodes:
            if isinstance(self.int_to_node[x], convolution_block):
                conv_blocks.append(x)
            elif isinstance(self.int_to_node[x], max_pool_node):
                max_pool_blocks.append(x)
        return (conv_blocks, max_pool_blocks)
    
    def topsorting(self):
        # level problem
        topsort = []
        import Queue
        in_deg = {}
        q = Queue.Queue()
        for node in self.nodes:
            val  = len(self.adj_list[node][0])
#             val = len(self.int_to_node[node].in_adj)
            if val == 0:
                q.put(node)
            in_deg[node] = val
            
        while not q.empty():
            curr_node = q.get()
            topsort.append(curr_node)
            for child in self.adj_list[curr_node][1]:
                in_deg[child] -= 1
                if in_deg[child] == 0:
                    q.put(child)
        return topsort
    
    def add_nodes_to_network(self, nodes):
        ### loophole here, assumption is that all changed nodes are being provided to the function
        ### for now, lets go on with it, but its an issue
        for curr_node in nodes:
            curr_node.determine_compatibility()
            if not curr_node.compatible:
                raise Error('Node is not compatible with the graph') 
        for curr_node in nodes:
            if curr_node not in self.node_to_int:
                self.max_no += 1
                self.adj_mat[self.max_no] = {}
                self.adj_list[self.max_no] = []
                self.node_to_int[curr_node] = self.max_no
                self.int_to_node[self.max_no] = curr_node
                self.nodes.add(self.max_no)
                if isinstance(curr_node, convolution_block):
                    self.conv_blocks.append(self.max_no)
                elif isinstance(curr_node, max_pool_node):
                    self.max_pool_blocks.append(self.max_no)
        for curr_node in nodes:
            no = self.node_to_int[curr_node]
            self.adj_list[no] = [map(lambda x: self.node_to_int[x], curr_node.in_adj), map(lambda x: self.node_to_int[x], curr_node.out_adj)]
            for par in self.adj_list[no][0]:
                self.adj_mat[par][no] = 1
            for child in self.adj_list[no][0]:
                self.adj_mat[no][child] = 1
        self.topsort = self.topsorting()
        
    def morphism(self):
        import random
        actions = {'deepen': self.deepen_morph, 
                   'widen': self.widen_morph, 
                   'skip-connection': self.skip_morph }
        choice = random.choice(actions)
        actions[choice]()
    
    def deepen_morph(self):
        deepen_conv_block = self.int_to_node[random.choice(self.conv_blocks)]
        kernel_size = random.choice([3, 5])
        in_channels, in_h, in_w = deepen_conv_block.output_shape
        out_channels = in_channels
        identity_conv_block = convolution_block(in_h, in_w, in_channels, out_channels, kernel_size, (kernel_size-1)/2)
        weights = identity_conv_block.conv_layer.weight.data
        
        # creating identity weights
        for channel in range(out_channels):
            for i in range(in_channels):
                for j in range(kernel_size):
                    for k in range(kernel_size):
                        weights[channel][i][j][k] = int((channel == i) and (j == k) and j == (kernel_size)/2 )
#         print 'weights of identity conv block', weights
        
        ## make connections 
        identity_conv_block.describe_adj_list([deepen_conv_block], deepen_conv_block.out_adj)
        deepen_conv_block.describe_adj_list(deepen_conv_block.in_adj, [identity_conv_block])

        #### later look at creating a function for singular change to in_adj or out_adj of nodes
        for out_node in identity_conv_block.out_adj:
            out_node_in_adj = [identity_conv_block if (x == deepen_conv_block) else x for x in out_node.in_adj ]
            out_node.describe_adj_list(out_node_in_adj, out_node.out_adj)
        
        self.add_nodes_to_network([deepen_conv_block, identity_conv_block] + identity_conv_block.out_adj)
    
    
    def widen_morph(self):
        candidate_conv_blocks = []
        for conv_block in self.conv_blocks:
            isCandidate = bool(len(self.adj_list[conv_block][1]))
            for child in self.adj_list[conv_block][1]:
                isCandidate = isCandidate and isinstance(self.int_to_node[child], convolution_block)
            if isCandidate:
                candidate_conv_blocks.append(conv_block)
        if len(candidate_conv_blocks) == 0:
            return False

        parent_block_no = random.choice(candidate_conv_blocks)
        print candidate_conv_blocks
        print parent_block_no
        parent_block = self.int_to_node[parent_block_no]
        widening_factor = random.choice([2, 4])
        in_channels, in_h, in_w = parent_block.input_shape
        out_channels = parent_block.out_channels
        kernel_size = parent_block.kernel_size
        padding = parent_block.padding
        stride = parent_block.stride
        widened_parent_block = convolution_block(in_h, in_w, in_channels, out_channels*widening_factor, kernel_size, padding, stride)
        original_parent_weight = parent_block.conv_layer.weight.data
        widened_parent_weight = widened_parent_block.conv_layer.weight.data
        widened_parent_weight[:out_channels] = original_parent_weight
        widened_parent_weight[out_channels:] = torch.zeros((out_channels*(widening_factor-1), in_channels, kernel_size, kernel_size))
        self.int_to_node[parent_block_no]  = widened_parent_block
        del self.node_to_int[parent_block]
        self.node_to_int[widened_parent_block] = parent_block_no
        parent_out_adj = []
        for child in parent_block.out_adj:
            child_no = self.node_to_int[child]
            in_channels, in_h, in_w = child.input_shape
            out_channels = child.out_channels
            kernel_size = child.kernel_size 
            padding = child.padding
            stride = child.stride
            child_widened = convolution_block(in_h, in_w, in_channels*widening_factor, out_channels, kernel_size, padding, stride)
            child_widened.conv_layer.weight.data[:, :in_channels, :, :] = child.conv_layer.weight.data
            child_widened.describe_adj_list([widened_parent_block if x == parent_block else x in child.in_adj], child.out_adj)
            self.int_to_node[child_no] = child_widened
            del self.node_to_int[child]
            self.node_to_int[child_widened] = child_no
            parent_out_adj.append(widened_child)
        widened_parent_block.describe_adj_list(parent_block.in_adj, parent_out_adj)
        
    def skip_morph(self):
        pass
    
    def visualize(self):
        graph = Digraph('arch', 'arch.gv')
        for no, curr_node in self.int_to_node.items():
#             graph.node(str(no), str(type(curr_node)).split('__main__.')[1])
            graph.node(str(no), str(self.node_to_int[curr_node]) + " :: " + repr(curr_node)[:200])
        for no, li in self.adj_list.items():
            for ch in li[1]:
                graph.edge(str(no), str(ch))
        graph.view()
    
    def describe(self):
        print 'Nodes: ', self.nodes
        print 'Conv_blocks', self.conv_blocks
        print 'Max_pool_blocks', self.max_pool_blocks
        print 'Adj_list', self.adj_list
        print 'Adj_mat', self.adj_mat
        print 'int_to_node', self.int_to_node
        print 'node_to_int', self.node_to_int
        print 'Toposort', self.topsort
    

In [46]:
dummy = node((5, 5, 5), (5, 5, 5))
n1 = convolution_block(5, 5, 5, 4, 3)
n2 = convolution_block(3, 3, 4, 3, 2)
dummy.describe_adj_list([], [n1])
n1.describe_adj_list([dummy], [n2])
n2.describe_adj_list([n1], [])

In [47]:
net = Network({0:[[], [1]], 1:[[0], [2]], 2: [[1], []]}, {0: dummy, 1:n1, 2:n2})
#### remove morph function from Network class to shift to hill climbing, also kernel_size and widening factor 
#### for deepen and widen should be parameters
### check more conformity betweeen input adj_list and input 'int_to_node'
### check compatibility of initial arch

In [48]:
net.describe()

Nodes:  set([0, 1, 2])
Conv_blocks [1, 2]
Max_pool_blocks []
Adj_list {0: [[], [1]], 1: [[0], [2]], 2: [[1], []]}
Adj_mat {0: {0: 0, 1: 1, 2: 0}, 1: {0: 0, 1: 0, 2: 1}, 2: {0: 0, 1: 0, 2: 0}}
int_to_node {0: <class '__main__.node'> 21, 1: convolution_block(
  (conv_layer): Conv2d(5, 4, kernel_size=(3, 3), stride=(1, 1))
  (batch_norm): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
), 2: convolution_block(
  (conv_layer): Conv2d(4, 3, kernel_size=(2, 2), stride=(1, 1))
  (batch_norm): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)}
node_to_int {convolution_block(
  (conv_layer): Conv2d(4, 3, kernel_size=(2, 2), stride=(1, 1))
  (batch_norm): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
): 2, <class '__main__.node'> 21: 0, convolution_block(
  (conv_layer): Conv2d(5, 4, kernel_size=(3, 3), stride=(1, 1))
  (batch_norm): BatchNorm2d(4, eps

In [49]:
net.visualize()
for nan in net.nodes:
    print net.int_to_node[nan].input_shape, ' -> ', nan, '->', net.int_to_node[nan].output_shape 

(5, 5, 5)  ->  0 -> (5, 5, 5)
(5, 5, 5)  ->  1 -> (4, 3, 3)
(4, 3, 3)  ->  2 -> (3, 2, 2)


In [50]:
net.widen_morph()

[1]
1


NameError: global name 'x' is not defined

In [31]:
net.describe()

Nodes:  set([0, 1, 2])
Conv_blocks [1, 2]
Max_pool_blocks []
Adj_list {0: [[], [1]], 1: [[0], [2]], 2: [[1], []]}
Adj_mat {0: {0: 0, 1: 1, 2: 0}, 1: {0: 0, 1: 0, 2: 1}, 2: {0: 0, 1: 0, 2: 0}}
int_to_node {0: <class '__main__.node'> 10, 1: convolution_block(
  (conv_layer): Conv2d(5, 4, kernel_size=(3, 3), stride=(1, 1))
  (batch_norm): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
), 2: convolution_block(
  (conv_layer): Conv2d(4, 12, kernel_size=(2, 2), stride=(1, 1))
  (batch_norm): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)}
node_to_int {<class '__main__.node'> 10: 0, convolution_block(
  (conv_layer): Conv2d(4, 12, kernel_size=(2, 2), stride=(1, 1))
  (batch_norm): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
): 2, convolution_block(
  (conv_layer): Conv2d(5, 4, kernel_size=(3, 3), stride=(1, 1))
  (batch_norm): BatchNorm2d(4,

In [32]:
net.visualize()
for nan in net.nodes:
    print net.int_to_node[nan].input_shape, ' -> ', nan, '->', net.int_to_node[nan].output_shape 

(5, 5, 5)  ->  0 -> (5, 5, 5)
(5, 5, 5)  ->  1 -> (4, 3, 3)
(4, 3, 3)  ->  2 -> (12, 2, 2)


In [24]:
www = [1]

In [ ]:
www.emp